# Fast Wavelet Transformation (FWT) Implementation For Maximum Decomposition Level with Haar and Daubechies Wavelets

In [ ]:
import numpy as np
from math import log, ceil, floor
import pywt   

In [ ]:
def nextPowerOf2(n): 
    count = 0; 
    if (n and not(n & (n - 1))): 
        return n       
    while( n != 0): 
        n >>= 1
        count += 1     
    return 1 << count;  

### Haar One Level

In [ ]:
def haarOneLevel(g):
    L = int(len(g)/2)
    G_low, G_high = np.zeros(L), np.zeros(L)
    for i in range(L):
        v0, v1 = g[2*i], g[2*i+1]
        G_low[i] = (v0 + v1)/np.sqrt(2.0)
        G_high[i] = (v0 - v1)/np.sqrt(2.0)
    return G_low, G_high

def haarOneLevelInverse(G):
    L = int(len(G))
    g = np.zeros((L))
    for i in range(L//2):
        v0, v1 = G[i], G[i+int(L/2)]
        g[2*i] = (v0 + v1)/np.sqrt(2.0)
        g[2*i+1] = (v0 - v1)/np.sqrt(2.0)
    return g

### Daubechies-4 (D4) Tap

In [ ]:
def db4(g):
    L = int(len(g))
    l0 = (1+np.sqrt(3))/(4*np.sqrt(2))
    l1 = (3+np.sqrt(3))/(4*np.sqrt(2))
    l2 = (3-np.sqrt(3))/(4*np.sqrt(2))
    l3 = (1-np.sqrt(3))/(4*np.sqrt(2))
    h0, h1, h2, h3 = l3, -l2, l1, -l0

    G_low, G_high = np.zeros(L//2), np.zeros(L//2)
    for i in range(0, len(g), 2):
        G_low[i//2] = g[i]*l0 + g[i+1]*l1 + g[(i+2)%L]*l2 + g[(i+3)%L]*l3
        G_high[i//2] = g[i]*h0 + g[i+1]*h1 + g[(i+2)%L]*h2 + g[(i+3)%L]*h3
   
    return G_low, G_high  

def db4Inverse(G):
    L = int(len(G))
    l0 = (1+np.sqrt(3))/(4*np.sqrt(2))
    l1 = (3+np.sqrt(3))/(4*np.sqrt(2))
    l2 = (3-np.sqrt(3))/(4*np.sqrt(2))
    l3 = (1-np.sqrt(3))/(4*np.sqrt(2))
    h0, h1, h2, h3 = l3, -l2, l1, -l0

    g = np.zeros((L))
    for i in range(0, len(g), 2):
        g[i] = G[i]*l0 + G[i+1]*l3 + G[(i-2)%L]*l2 + G[(i-1)%L]*l1
        g[i+1] = G[i]*h2 + G[i+1]*h1 + G[(i-2)%L]*h0 + G[(i-1)%L]*h3  
    return g


# Test
cA_level1, cD_level1 = db4(np.array([1,1,4,4,0,0,1,1]))
print(cA_level1)
print(cD_level1)
cA_level2, cD_level2 = db4(cA_level1)
print(cA_level2)
# returns cA_level1
print(db4Inverse([cA_level2[0], cD_level2[0], cA_level2[1], cD_level2[1]]))
# returns original signal
print(db4Inverse([cA_level1[0], cD_level1[0], cA_level1[1], cD_level1[1], 
                  cA_level1[2], cD_level1[2], cA_level1[3], cD_level1[3]]))

fwt implementation using recursion

In [ ]:
def fwt(g, mode='haar'):
    if mode == 'haar':
        G_low, G_high = haarOneLevel(g)
        if len(g) == 2:
            return np.concatenate((G_low, G_high))
        else:
            return np.concatenate((fwt(G_low), G_high))  
    
    if mode == 'db4':
        G_low, G_high = db4(g)
        if len(g) == 4:
            return np.concatenate((G_low, G_high))
        else:
            return np.concatenate((fwt(G_low, 'db4'), G_high)) 

In [ ]:
def ifwt(G, mode='haar'):
    if mode == 'haar':
        L = int(log(len(G),2))
        first_half = G[:2**1]
        for i in range(1, L):
            first_half = haarOneLevelInverse(first_half)
            second_half = G[2**i:2**(i+1)]   
            first_half = np.concatenate((first_half, second_half))
        return haarOneLevelInverse(first_half)
    
    if mode == 'db4':
        L = int(log(len(G),2))-1
        first_half = [G[i] for i in [0,2,1,3]]
        for i in range(1, L):
            first_half = db4Inverse(first_half)
            second_half = G[4**i:4**(i+1)] 
            temp = np.zeros(2*len(first_half))
            for i, (x, y) in enumerate(zip(first_half, second_half)):
                temp[2*i] = x
                temp[2*i+1] = y
            first_half = temp
        return db4Inverse(first_half)

In [ ]:
# Function to check 
# Log base 2 
def Log2(x): 
    if x == 0: 
        return false; 
  
    return (log(x) / log(2)); 
  
# Function to check 
# if x is power of 2 
def isPowerOfTwo(n): 
    return (ceil(Log2(n)) == floor(Log2(n)))

### Demo - Power of 2 (Haar)

In [ ]:
g = np.array([1,7,3,0,5,4,2,9])
G = fwt(g)
G

In [ ]:
ifwt(G)

### Demo - Nearest Power of 2 (Haar)

In [ ]:
g = np.array([1,7,3,0,5,2,0])
L = len(g)
if isPowerOfTwo(L) == False:
    num_pads = nextPowerOf2(L) - L
    g = np.concatenate((g, np.zeros(num_pads)))
else:
    num_pads = 0

In [ ]:
G = fwt(g)
G[:] if num_pads == 0 else G[:-num_pads]

In [ ]:
g = ifwt(G)
g[:-num_pads]

In [ ]:
cA, cD = pywt.dwt([1,7,3,0,5,2,0], 'haar')
print(cA, cD)

In [ ]:
cA, cD = pywt.dwt([1,7,3,0,5,2,0], 'haar')
print(cA, cD)

### Demo - Nearest Power of 2 (D4)

In [ ]:
g = np.array([1,1,4,4,0,0,1,1])
L = len(g)
if isPowerOfTwo(L) == False:
    num_pads = nextPowerOf2(L) - L
    g = np.concatenate((g, np.zeros(num_pads)))
else:
    num_pads = 0

In [ ]:
G = fwt(g, 'db4')
G[:] if num_pads == 0 else G[:-num_pads]
# G_ll[0], G_ll[1], G_hl[0], G_hl[1], G_h[0], G_h[1], G_h[2], G_h[3]

In [ ]:
g = ifwt(G, 'db4')
g[:] if num_pads == 0 else g[:-num_pads]